# Running Palace Simulations: Mach Zehnder Modulator

[Palace](https://awslabs.github.io/palace/) is an open-source 3D electromagnetic simulator supporting eigenmode, driven (S-parameter), and electrostatic simulations. This notebook demonstrates using the `gsim.palace` API to run a driven simulation on a Mach Zehnder Modulator.

**Requirements:**

- [GDSFactory+](https://gdsfactory.com) account for cloud simulation

### Construct custom layer stack

In [ ]:
from pathlib import Path
import gdsfactory as gf
from gsim.common import LayerStack, Layer
from gsim.common import MATERIALS_DB

CONDUCTOR_THICKNESS=1
QUARTZ_THICKNESS=300
SILICON_DIOXIDE_THICKNESS=2
TFLN_BOTTOM_THICKNESS=0.6
TFLN_TOP_THICKNESS=0.3
AIR_THICKNESS=300

quartz_layer = Layer(
    name="quartz_substrate",
    gds_layer=(1, 0),
    zmin=0,
    zmax=QUARTZ_THICKNESS,
    thickness=QUARTZ_THICKNESS,
    material="quartz",
    layer_type="dielectric",
    mesh_resolution="coarse"
)
siO2_layer = Layer(
    name="silicon_dioxide",
    gds_layer=(2, 0),
    zmin=quartz_layer.zmax,
    zmax=quartz_layer.zmax+SILICON_DIOXIDE_THICKNESS,
    thickness=SILICON_DIOXIDE_THICKNESS,
    material="SiO2",
    layer_type="dielectric",
    mesh_resolution="coarse"
)

tfln_base_layer = Layer(
    name="tfln_base",
    gds_layer=(3, 0),
    zmin=siO2_layer.zmax,
    zmax=siO2_layer.zmax+TFLN_BOTTOM_THICKNESS,
    thickness=TFLN_BOTTOM_THICKNESS,
    material="tfln",
    layer_type="dielectric",
    mesh_resolution="coarse"
)
tfln_rib_layer = Layer(
    name="tfln_rib",
    gds_layer=(4, 0),
    zmin=tfln_base_layer.zmax,
    zmax=tfln_base_layer.zmax+TFLN_TOP_THICKNESS,
    thickness=TFLN_TOP_THICKNESS,
    material="tfln",
    layer_type="conductor",
    mesh_resolution="coarse"
)
gold_layer = Layer(
    name="gold",
    gds_layer=(5, 0),
    zmin=tfln_base_layer.zmax,
    zmax=tfln_base_layer.zmax+CONDUCTOR_THICKNESS,
    thickness=CONDUCTOR_THICKNESS,
    material="gold",
    layer_type="conductor",
    mesh_resolution="fine"
)
layers = [quartz_layer, siO2_layer, tfln_base_layer, tfln_rib_layer, gold_layer]

# Create layer stack and export to YAML
layer_stack = LayerStack.from_layer_list(layers)
stack_filename = "mzm_stack.yaml"
layer_stack.to_yaml(Path(stack_filename))

### Generate layout for MZM

In [ ]:
gf.gpdk.PDK.activate()


def t_shape(
    name: str, layer: tuple[int, int], w1: float, w2: float, h1: float, h2: float
) -> gf.Component:
    t = gf.Component(name)
    w3 = (w2 - w1) / 2
    t.add_polygon(
        [
            (-w1 / 2, 0),
            (w1 / 2, 0),
            (w1 / 2, h1),
            (w1 / 2 + w3, h1),
            (w1 / 2 + w3, h1 + h2),
            (-w1 / 2 - w3, h1 + h2),
            (-w1 / 2 - w3, h1),
            (-w1 / 2, h1),
        ],
        layer=layer,
    )
    return t


def cpw_conductor(
    layer: tuple[int, int],
    board_length: float,
    board_width: float,
    gap: float,
    trace_width: float,
    ground_width: float,
    w1: float,
    w2: float,
    h1: float,
    h2: float,
    t_spacing: float,
) -> gf.Component:
    c = gf.Component("cpw_conductor")
    # Draw conductor trace
    midwidth = board_width / 2
    # Determine how many Ts we can draw
    number_of_ts = int(board_length / (w2 + t_spacing))
    leftover_length = board_length - number_of_ts * (w2 + t_spacing)
    x_start = leftover_length / 2

    for i in range(number_of_ts):
        t = t_shape(name=f"t_shape_g1l{i}", layer=layer, w1=w1, w2=w2, h1=h1, h2=h2)
        t_ref = c.add_ref(t)
        t_ref.move(
            (x_start + i * (w2 + t_spacing) + w2 / 2, midwidth + trace_width / 2)
        )
    for i in range(number_of_ts):
        t = t_shape(name=f"t_shape_g1u{i}", layer=layer, w1=w1, w2=w2, h1=h1, h2=h2)
        t.rotate(180)
        t_ref = c.add_ref(t)
        t_ref.move(
            (x_start + i * (w2 + t_spacing) + w2 / 2, midwidth + trace_width / 2 + gap)
        )
    c.add_polygon(
        [
            (0, midwidth - trace_width / 2),
            (board_length, midwidth - trace_width / 2),
            (board_length, midwidth + trace_width / 2),
            (0, midwidth + trace_width / 2),
        ],
        layer=layer,
    )
    for i in range(number_of_ts):
        t = t_shape(name=f"t_shape_g2u{i}", layer=layer, w1=w1, w2=w2, h1=h1, h2=h2)
        t.rotate(180)
        t_ref = c.add_ref(t)
        t_ref.move(
            (x_start + i * (w2 + t_spacing) + w2 / 2, midwidth - trace_width / 2)
        )
    for i in range(number_of_ts):
        t = t_shape(name=f"t_shape_g2l{i}", layer=layer, w1=w1, w2=w2, h1=h1, h2=h2)
        t_ref = c.add_ref(t)
        t_ref.move(
            (x_start + i * (w2 + t_spacing) + w2 / 2, midwidth - trace_width / 2 - gap)
        )

    # Draw ground planes
    c.add_polygon(
        [
            (0, midwidth - trace_width / 2 - gap),
            (0, midwidth - trace_width / 2 - gap - ground_width),
            (board_length, midwidth - trace_width / 2 - gap - ground_width),
            (board_length, midwidth - trace_width / 2 - gap),
        ],
        layer=layer,
    )
    c.add_polygon(
        [
            (0, midwidth + trace_width / 2 + gap),
            (0, midwidth + trace_width / 2 + gap + ground_width),
            (board_length, midwidth + trace_width / 2 + gap + ground_width),
            (board_length, midwidth + trace_width / 2 + gap),
        ],
        layer=layer,
    )
    return c


def rib(
    layer: tuple[int, int],
    board_width: float,
    board_length: float,
    gap: float,
    trace_width: float,
    opt_width: float,
) -> gf.Component:
    rib_comp = gf.Component("rib")
    midwidth = board_width / 2
    center = midwidth + trace_width / 2 + gap / 2
    rib_comp.add_polygon(
        [
            (0, center - opt_width / 2),
            (board_length, center - opt_width / 2),
            (board_length, center + opt_width / 2),
            (0, center + opt_width / 2),
        ],
        layer=layer,
    )
    center = midwidth - trace_width / 2 - gap / 2
    rib_comp.add_polygon(
        [
            (0, center - opt_width / 2),
            (board_length, center - opt_width / 2),
            (board_length, center + opt_width / 2),
            (0, center + opt_width / 2),
        ],
        layer=layer,
    )
    return rib_comp


def mzm_cpw(
    name: str,
    quartz_layer: Layer,
    siO2_layer: Layer,
    tfln_base_layer: Layer,
    tfln_rib_layer: Layer,
    conductor_layer: Layer,
    board_width: float,
    board_length: float,
    gap: float,
    trace_width: float,
    ground_width: float,
    rib_width: float,
    t_w1: float,
    t_w2: float,
    t_h1: float,
    t_h2: float,
    t_spacing: float,
    port_length: float
) -> gf.Component:
    comp = gf.Component(name)
    comp.add_polygon(
        [
            (0, 0),
            (board_length, 0),
            (board_length, board_width),
            (0, board_width),
        ],
        layer=quartz_layer.gds_layer,
    )
    comp.add_polygon(
        [
            (0, 0),
            (board_length, 0),
            (board_length, board_width),
            (0, board_width),
        ],
        layer=siO2_layer.gds_layer,
    )
    comp.add_polygon(
        [
            (0, 0),
            (board_length, 0),
            (board_length, board_width),
            (0, board_width),
        ],
        layer=tfln_base_layer.gds_layer,
    )
    cpw_conductor_comp = cpw_conductor(
        layer=conductor_layer.gds_layer,
        board_length=board_length,
        board_width=board_width,
        gap=gap,
        trace_width=trace_width,
        ground_width=ground_width,
        w1=t_w1,
        w2=t_w2,
        h1=t_h1,
        h2=t_h2,
        t_spacing=t_spacing,
    )
    comp.add_ref(cpw_conductor_comp)

    rib_comp = rib(
        layer=tfln_rib_layer.gds_layer,
        board_width=board_width,
        board_length=board_length,
        gap=gap,
        trace_width=trace_width,
        opt_width=rib_width,
    )
    comp.add_ref(rib_comp)

    midwidth = board_width / 2

    comp.add_port(
        name="P1",
        center=(port_length/2, midwidth + trace_width / 2 + gap / 2),
        width=gap,
        orientation=0,
        port_type="electrical",
        layer=conductor_layer.gds_layer,
    )
    comp.add_port(
        name="P2",
        center=(port_length/2, midwidth - trace_width / 2 - gap / 2),
        width=gap,
        orientation=0,
        port_type="electrical",
        layer=conductor_layer.gds_layer,
    )
    comp.add_port(
        name="P3",
        center=(board_length-port_length/2, midwidth + trace_width / 2 + gap / 2),
        width=gap,
        orientation=180,
        port_type="electrical",
        layer=conductor_layer.gds_layer,
    )
    comp.add_port(
        name="P4",
        center=(board_length-port_length/2, midwidth - trace_width / 2 - gap / 2),
        width=gap,
        orientation=180,
        port_type="electrical",
        layer=conductor_layer.gds_layer,
    )
    return comp

# T-shape parameters
w1=2 # width of the stem of the T
w2=45 # width of the T top bar
h1=6 # height of the stem of the T
h2=2 # height of the T top bar
spacing=5 # spacing between Ts
MZM_CPW_GAP = 5+2*(h1+h2) # gap + 2*(h1+h2) to ensure ports are outside of the T shapes
PORT_LENGTH = MZM_CPW_GAP/2 # length of the port to ensure it's outside of the T shapes

mzm_cpw_comp = mzm_cpw(
    "mzm_cpw",
    quartz_layer=quartz_layer,
    siO2_layer=siO2_layer,
    tfln_base_layer=tfln_base_layer,
    tfln_rib_layer=tfln_rib_layer,
    conductor_layer=gold_layer,
    board_length=2075,
    board_width=720,
    gap=MZM_CPW_GAP,
    trace_width=100,
    ground_width=300,
    rib_width=1,
    t_w1=w1,
    t_w2=w2,
    t_h1=h1,
    t_h2=h2,
    t_spacing=spacing,
    port_length=PORT_LENGTH
)

# Draw and plot MZM CPW component
_mzm_cpw_comp = mzm_cpw_comp.copy()
_mzm_cpw_comp.draw_ports()
_mzm_cpw_comp.plot()


### Configure and run simulation with DrivenSim

In [ ]:
from gsim.palace import DrivenSim

# Create simulation object
sim = DrivenSim()

# Set output directory
sim.set_output_dir("./palace-sim-mzm")

# Set the component geometry
sim.set_geometry(mzm_cpw_comp)

# Configure layer stack from active PDK
sim.set_stack(yaml_path=Path(stack_filename))

# Configure via ports (Metal1 ground plane to TopMetal2 signal)
# Configure left CPW port (P1 lower, P2 upper)
sim.add_cpw_port("P2", "P1", layer="gold", length=PORT_LENGTH)

# Configure right CPW port (P4 lower, P3 upper)
sim.add_cpw_port("P3", "P4", layer="gold", length=PORT_LENGTH)

# Configure driven simulation (frequency sweep for S-parameters)
sim.set_driven(fmin=1e9, fmax=100e9, num_points=40)

# Validate configuration
print(sim.validate_config())

In [ ]:
# Generate mesh (presets: "coarse", "default", "fine")
sim.mesh(preset="coarse", margin=15, air_above=50)

In [ ]:
# Interactive
#sim.plot_mesh(show_groups=["gold_xy", "gold_z", "P1_E0", "P1_E1", "P2_E0", "P2_E1"], interactive=True)
sim.plot_mesh(interactive=True)

In [ ]:
# Generate Palace config file
sim.write_config()

### Run simulation on GDSFactory+ Cloud

In [ ]:
# Run simulation on GDSFactory+ cloud
results = sim.simulate()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(results["port-S.csv"])
df.columns = df.columns.str.strip()  # Remove whitespace from column names

freq = df["f (GHz)"]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))

# Magnitude plot
ax1.plot(freq, df["|S[1][1]| (dB)"], marker=".", label="S11")
ax1.plot(freq, df["|S[2][1]| (dB)"], marker=".", label="S21")
ax1.set_xlabel("Frequency (GHz)")
ax1.set_ylabel("Magnitude (dB)")
ax1.set_title("S-Parameters")
ax1.legend()
ax1.grid(True)

# Phase plot
ax2.plot(freq, df["arg(S[1][1]) (deg.)"], marker=".", label="S11")
ax2.plot(freq, df["arg(S[2][1]) (deg.)"], marker=".", label="S21")
ax2.set_xlabel("Frequency (GHz)")
ax2.set_ylabel("Phase (deg)")
ax2.legend()
ax2.grid(True)

plt.tight_layout()